In [1]:
import itertools
import sys, os

import numpy as np
import pandas as pd
from scipy.special import comb
from scipy import stats
import scipy.cluster.hierarchy as hac
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

from collections import Counter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

sns.set(rc={'figure.figsize':(15,8)})
sns.set_context('poster')

In [2]:
%load_ext autoreload
%autoreload 2
#import CCPA_lib as cp


In [3]:
sns.set_context('poster')


![](http://oregonstate.edu/instruct/bb450/fall14/stryer7/2/table_02_02.jpg)

https://www.uniprot.org/uniprot/Q7V735

http://tigrfams.jcvi.org/cgi-bin/HmmReportPage.cgi?acc=TIGR03798

https://www.ebi.ac.uk/training/online/course/interpro-functional-and-structural-analysis-protei/sequence-searching/searching-interpro-batc

http://www.ebi.ac.uk/interpro/sequencesearch/iprscan5-S20190707-131508-0462-76111813-p1m

https://www.ebi.ac.uk/Tools/services/rest/iprscan5/result/iprscan5-S20190707-131508-0462-76111813-p1m/json

In [47]:
import subprocess
import os
#-in ../../rnaseq_mapping/mit9313/MIT9313.fasta  -g 11 -ml 1 
#-out MIT9313.orffinder.out -outfmt 1 -s 0
def run_orffinder(accession, minimum_length='1', 
                  dna_code='11', atg_only=True):
    
    cwd = os.getcwd()
    orffinder_exe = os.path.join(cwd, 'bin','ORFfinder')
    out_dpath = os.path.join(cwd,'orffinder_tmp')
    out_fpath = os.path.join(out_dpath, f'{accession}.orffinder.fa')
    
    
    params = [orffinder_exe, 
              '-id', accession, 
              '-g', dna_code,
              '-ml', minimum_length,
              '-s', '0', # ATG only start codon
              '-out', out_fpath,              
    ]
    
    #print(params)
    try:
        os.makedirs(out_dpath, exist_ok=True)
        print(' '.join(params))
        retcode = subprocess.check_call(' '.join(params), shell=True)
        if retcode < 0:
            print("Child was terminated by signal", -retcode)
    except OSError as e:
        print("Execution failed:", e)
    except subprocess.CalledProcessError as e:
        print("Execution failed:", e)
    return out_fpath
    


In [19]:
run_orffinder('BX548174', minimun_length='1000')

/home/osnat/short_pedtides/ripp/bin/ORFfinder -id BX548174 -g 11 -ml 1000 -s 0 -out /home/osnat/short_pedtides/ripp/orffinder_tmp/BX548174.orffinder.fa


'/home/osnat/short_pedtides/ripp/orffinder_tmp/BX548174.orffinder.fa'

In [105]:
#name = '>lcl|ORF1_BX548174.1:2043:4382 unnamed protein product'
#name = '>lcl|ORF521_BX548174.1:17683:16679 unnamed protein product'
def _parse_orf_name(name):
    n=name.split()[0]
    n=n.split('|')[1]
    geneid, start, stop = n.split(':')
    contig = geneid.split('_', 2)[1]
    start = int(start)
    stop = int(stop)
    l = start
    r= stop
    strand = '+'
    if l >= r:
        strand = '-'
        l = stop
        r = start
    return {
        'orfid' : n,
        'contig' : contig,
        'left' : l,
        'right' : r,
        'start' : start,
        'stop' : stop,
        'strand' : strand,
    }


In [106]:
_parse_orf_name('>lcl|ORF521_BX548174.1:17683:16679 unnamed protein product')

{'orfid': 'ORF521_BX548174.1:17683:16679',
 'contig': 'BX548174.1',
 'left': 16679,
 'right': 17683,
 'start': 17683,
 'end': 16679,
 'strand': '-'}

In [107]:
_parse_orf_name('>lcl|ORF1_BX548174.1:2043:4382 unnamed protein product')

{'orfid': 'ORF1_BX548174.1:2043:4382',
 'contig': 'BX548174.1',
 'left': 2043,
 'right': 4382,
 'start': 2043,
 'end': 4382,
 'strand': '+'}

In [124]:
def list_orfs(accession, minimum_length='1000'):
    orf_fpath = run_orffinder(accession, minimum_length=minimum_length)
    result = list()
    cur_item = None
    cur_aaseq = list()
    with open(orf_fpath, 'r') as fh:
        for l in fh:
            l = l.strip()
            #print (l)
            if l.startswith('>'):
                # new item
                #>lcl|ORF1_BX548174.1:2043:4382 unnamed protein product
                if cur_item is not None:
                    cur_item['aaseq'] = ''.join(cur_aaseq)
                    result.append(cur_item)
                cur_aaseq = list()
                cur_item = _parse_orf_name(l)
            else:
                cur_aaseq.append(l)
    if cur_item is not None:
        cur_item['aaseq'] = ''.join(cur_aaseq)
        result.append(cur_item)

    cols = ['orfid', 'contig', 'left', 'right', 
            'start', 'stop', 'strand', 'aaseq', ]
    df = pd.DataFrame(result, columns=cols)
    for c in ['stop', 'left','right','start']:
        df[c] = pd.to_numeric(df[c])
    df['len_nn'] = (df.right + 1 - df.left) 
    df['len_aa'] = df['aaseq'].str.len()
    df['len_aa3'] = (df['len_aa']+1)*3
    df['rast_left'] = df['left'] + 1
    df['rast_right'] = df['right'] + 1
    return df
    

In [125]:
df = list_orfs('BX548174')
df.head()

/home/osnat/short_pedtides/ripp/bin/ORFfinder -id BX548174 -g 11 -ml 1000 -s 0 -out /home/osnat/short_pedtides/ripp/orffinder_tmp/BX548174.orffinder.fa


,orfid,contig,left,right,start,end,strand,aaseq,len_nn,len_aa,len_aa3
0,ORF1_BX548174.1:2043:4382,BX548174.1,2043,4382,2043,4382,+,MIDSSSNNTYDVNESLKVENLTRDDYEEICKRLGRKPNRTELGMFG...,2340,779,2340
1,ORF2_BX548174.1:13470:14525,BX548174.1,13470,14525,13470,14525,+,MDDEIKSYFNNFDFSKNFKLKDDISFEKVLNNQFKEYVIKSYKVLS...,1056,351,1056
2,ORF3_BX548174.1:44304:46964,BX548174.1,44304,46964,44304,46964,+,MKSQTKNTPITGDEIRKEFLNFYHEKLHKIIPSASLIPDDPTVMLT...,2661,886,2661
3,ORF4_BX548174.1:72447:74375,BX548174.1,72447,74375,72447,74375,+,MNNAKNLKIKQIDKKNISFKELSLIKNIIFWVDIIPGDQTQKNAIF...,1929,642,1929
4,ORF5_BX548174.1:86775:87959,BX548174.1,86775,87959,86775,87959,+,MKSIKKPIVIVGAGFGGMTVASNLKELNPSLPILVVDSEAKFIFKP...,1185,394,1185


In [131]:
df.loc[df.len_aa3 != df.len_nn]

,orfid,contig,left,right,start,end,strand,aaseq,len_nn,len_aa,len_aa3,rast_left,rast_right


In [132]:
def load_genome(genome):
    genomes_dpath = os.path.join('.', 'data', 
                                 'detailed_Prochlorococcus_genome_annotations')
    fname = f'{genome}.txt'
    df = pd.read_csv(os.path.join(genomes_dpath, fname), sep='\t')
    df['genome'] = os.path.basename(os.path.splitext(fname)[0])
    df['left'] = gdf['start']
    df['right'] = gdf['stop']
    df.loc[gdf.strand == '-', 'left']  = df.loc[gdf.strand == '-', 'stop']
    df.loc[gdf.strand == '-', 'right'] = df.loc[gdf.strand == '-', 'start']

    return df


In [118]:
os.getcwd()

'/home/osnat/short_pedtides/ripp'

In [119]:
gdf = load_genome('MED4')

In [128]:
gdf.head()

,contig_id,gene_id,feature_id,type,location,start,stop,strand,function,aliases,figfam,evidence_codes,nucleotide_sequence,aa_sequence,genome
0,MED4,PMED4_0001,fig|1218.84.peg.1,peg,MED4_174_1331,174,1331,+,DNA polymerase III beta subunit (EC 2.7.7.7),NaN,FIG00066425,isu;DNA-replication isu;DNA_replication_cluster_1,atggagattgtttgtaatcaaaatgaatttaattatgctattcaat...,MEIVCNQNEFNYAIQLVSKAVASRPTHPILANLLLTADQGTNKISL...,MED4
1,MED4,PMED4_0002,fig|1218.84.peg.2,peg,MED4_1333_2040,1333,2040,+,hypothetical protein,NaN,NaN,NaN,ttgaaattaccaaaagaatttttattaagtgaattattaagacata...,MKLPKEFLLSELLRHNVKGNATLNYGKGENVWMHPPVHRILGWYSR...,MED4
2,MED4,PMED4_0003,fig|1218.84.peg.3,peg,MED4_2044_4383,2044,4383,+,"Phosphoribosylformylglycinamidine synthase, sy...",NaN,FIG01303876,icw(1);De_Novo_Purine_Biosynthesis,atgatagattcttcaagtaacaatacttatgatgttaatgaatcac...,MIDSSSNNTYDVNESLKVENLTRDDYEEICKRLGRKPNRTELGMFG...,MED4
3,MED4,PMED4_0004,fig|1218.84.peg.4,peg,MED4_4430_5890,4430,5890,+,Amidophosphoribosyltransferase (EC 2.4.2.14),NaN,FIG00000179,isu;YgfZ isu;De_Novo_Purine_Biosynthesis,atgtgtggaattgttggaattgtttcttgtgatgatgtaaatcaac...,MCGIVGIVSCDDVNQQIYDSLLLLQHRGQDSTGIATMENTVFHIHK...,MED4
4,MED4,PMED4_0005,fig|1218.84.peg.5,peg,MED4_8259_5887,8259,5887,-,DNA gyrase subunit A (EC 5.99.1.3),NaN,FIG00000080,idu(1);DNA_gyrase_subunits idu(1);DNA_topoisom...,atgagtgtgataataggacgtgctttacctgatgcaagagatggac...,MSVIIGRALPDARDGLKPVQRRILFAMHELGLTPDRPFRKCARVVG...,MED4


In [134]:
df.loc[df.left < 200]['aaseq']

165    MEIVCNQNEFNYAIQLVSKAVASRPTHPILANLLLTADQGTNKISL...
Name: aaseq, dtype: object

In [143]:
df.columns

Index(['orfid', 'contig', 'left', 'right', 'start', 'end', 'strand', 'aaseq',
       'len_nn', 'len_aa', 'len_aa3', 'rast_left', 'rast_right'],
      dtype='object')

In [138]:
gdf.loc[gdf.start < 200]['aa_sequence'].unique() == df.loc[df.left < 200]['aaseq'].unique()

array([ True])

In [140]:
gdf['left'] = gdf['start']
gdf['right'] = gdf['stop']


In [141]:
gdf.loc[gdf.strand == '-', 'left'] = gdf.loc[gdf.strand == '-', 'stop']
gdf.loc[gdf.strand == '-', 'right'] = gdf.loc[gdf.strand == '-', 'start']


In [146]:
df_merge = df.merge(right=gdf, how='left', 
         left_on=['rast_left', 'rast_right'],
         right_on=['left', 'right'],
         suffixes=['','_rast']
        )

In [149]:
df_merge.head().T

,0,1,2,3,4
orfid,ORF1_BX548174.1:2043:4382,ORF2_BX548174.1:13470:14525,ORF3_BX548174.1:44304:46964,ORF4_BX548174.1:72447:74375,ORF5_BX548174.1:86775:87959
contig,BX548174.1,BX548174.1,BX548174.1,BX548174.1,BX548174.1
left,2043,13470,44304,72447,86775
right,4382,14525,46964,74375,87959
start,2043,13470,44304,72447,86775
end,4382,14525,46964,74375,87959
strand,+,+,+,+,+
aaseq,MIDSSSNNTYDVNESLKVENLTRDDYEEICKRLGRKPNRTELGMFG...,MDDEIKSYFNNFDFSKNFKLKDDISFEKVLNNQFKEYVIKSYKVLS...,MKSQTKNTPITGDEIRKEFLNFYHEKLHKIIPSASLIPDDPTVMLT...,MNNAKNLKIKQIDKKNISFKELSLIKNIIFWVDIIPGDQTQKNAIF...,MKSIKKPIVIVGAGFGGMTVASNLKELNPSLPILVVDSEAKFIFKP...
len_nn,2340,1056,2661,1929,1185
len_aa,779,351,886,642,394


In [150]:
gdf.shape

(1999, 17)

In [152]:
df_merge.loc[~df_merge.genome.isna()].shape

(386, 30)